Source: https://github.com/Reinalynn/Building-a-Book-Recommendation-System-using-Python/blob/master/Code/Collab%20filtering%20using%20PySpark%20(user-based%20recommendations).ipynb

### 1. Import libraries

In [1]:
import os

import numpy as np
import pandas as pd

from pyspark.sql.functions import *
from pyspark.sql.types import *

from pyspark.ml.recommendation import ALS, ALSModel
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession

from pyspark.mllib.evaluation import RegressionMetrics, RankingMetrics
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator

from pyspark import SparkFiles

In [2]:
spark = SparkSession.builder.master("local").getOrCreate()  

### 2. Read datasets

In [4]:
# Read books dataset which contains various variables related to books only

books_df = spark.read.csv('books.csv', header = True, inferSchema=True)
books_df.show(5)

+-------+-----------------+------------+-------+-----------+---------+----------------+--------------------+-------------------------+--------------------+--------------------+-------------+--------------+-------------+------------------+-----------------------+---------+---------+---------+---------+---------+--------------------+--------------------+
|book_id|goodreads_book_id|best_book_id|work_id|books_count|     isbn|          isbn13|             authors|original_publication_year|      original_title|               title|language_code|average_rating|ratings_count|work_ratings_count|work_text_reviews_count|ratings_1|ratings_2|ratings_3|ratings_4|ratings_5|           image_url|     small_image_url|
+-------+-----------------+------------+-------+-----------+---------+----------------+--------------------+-------------------------+--------------------+--------------------+-------------+--------------+-------------+------------------+-----------------------+---------+---------+--------

In [6]:
# Read ratings dataset which contains ids of user and books and what ratings were assigned to various boos by users

ratings_df = spark.read.csv('ratings.csv', header = True, inferSchema=True)
ratings_df.show(5)

+-------+-------+------+
|user_id|book_id|rating|
+-------+-------+------+
|      1|    258|     5|
|      2|   4081|     4|
|      2|    260|     5|
|      2|   9296|     5|
|      2|   2318|     3|
+-------+-------+------+
only showing top 5 rows



### 3. A quick look into the dataset

In [11]:
total_records = ratings_df.select("rating").count()
total_distinct_users = ratings_df.select("user_id").distinct().count()
total_distinct_books = ratings_df.select("book_id").distinct().count()

print("Total records: {} \nTotal distint users: {} \nTotal distinct books: {} \n".format(total_records,total_distinct_users,total_distinct_books))

Total records: 230594 
Total distint users: 3788 
Total distinct books: 5743 



In [39]:
ratings_df.show(5)

+-------+-------+------+
|user_id|book_id|rating|
+-------+-------+------+
|      1|    258|   5.0|
|      2|   4081|   4.0|
|      2|    260|   5.0|
|      2|   9296|   5.0|
|      2|   2318|   3.0|
+-------+-------+------+
only showing top 5 rows



In [17]:
ratings_df.select("book_id").distinct().count()

5743

In [34]:
print("On average each book is reviewed by __ users: ")
ratings_df.groupBy("book_id").count().select(avg("count")).show()


On average each book is reviewed by __ users: 
+-----------------+
|       avg(count)|
+-----------------+
|40.15218526902316|
+-----------------+



In [37]:
print("On average each user reviewed by __ books: ")
ratings_df.groupBy("user_id").count().select(avg("count")).show()

On average each user reviewed by __ books: 
+-----------------+
|       avg(count)|
+-----------------+
|60.87486800422386|
+-----------------+



In [38]:
ratings_df = ratings_df.select(ratings_df.user_id.cast("integer"),
                                        ratings_df.book_id.cast("integer"),
                                        ratings_df.rating.cast("float"))
ratings_df.printSchema()

root
 |-- user_id: integer (nullable = true)
 |-- book_id: integer (nullable = true)
 |-- rating: float (nullable = true)



### 4. Data pre-processing

Using cross join to assign ratings for each book by each user. If a user did not rated a book, it will be marked as 0

In [56]:
users = ratings_df.select("user_id").distinct()
books = ratings_df.select("book_id").distinct()

In [58]:
# Cross join users and products

cj = users.crossJoin(books) # Cross joins return the cartesian product of two dataframes

ratings_df = cj.join(ratings_df, ["user_id", "book_id"], "left").fillna(0) # Fill na will mark any records count with 0, for which no review score exists
   
ratings_df.show(5)

+-------+-------+------+
|user_id|book_id|rating|
+-------+-------+------+
|    463|    471|   4.0|
|    463|    148|   0.0|
|    463|   2142|   0.0|
|    463|   3997|   0.0|
|    463|    496|   0.0|
+-------+-------+------+
only showing top 5 rows



In [59]:
ratings_df.count()

21754484

In [60]:
(train, test) = ratings_df.randomSplit([0.80, 0.20], seed=731)

In [63]:
train.count()       # About 17.4 Million records

17404073

In [64]:
test.count()        # About 4.3 Million records

4350411

### 5. Data Modelling

In [65]:
als_model = ALS(userCol = "user_id", itemCol = "book_id", ratingCol = "rating",     # This row simply defines all three columns for ALS algorithm
               nonnegative = True,                                                  # Enables non-negative constraints
               coldStartStrategy = "drop",                                          # Drop any values in dataframe which contain a NaN
               implicitPrefs = False)                                               # To use explicit feedback data

In [ ]:
# Fit the model

model = als_model.fit(train)
predictions = model.transform(test)

In [ ]:
evaluator = RegressionEvaluator(metricName = 'rmse', labelCol = 'rating',
                               predictionCol = 'prediction')
rmse = evaluator.evaluate(predictions)
print("RMSE: {}".format(rmse))

In [11]:
predictions.show(5)

+-------+-------+------+------------+
|user_id|book_id|rating|  prediction|
+-------+-------+------+------------+
|   1645|    148|   4.0|  0.05408534|
|   3997|    148|   0.0|0.0010495055|
|   4818|    148|   0.0| 0.028773755|
|   5518|    148|   0.0|  0.04367373|
|   1460|    148|   0.0| 0.084378034|
+-------+-------+------+------------+
only showing top 5 rows



In [12]:
# tweak model by playing with rank, MaxIter, RegParam, goal = lowest RMSE

In [13]:
# change rank only (chose 16 b/c it was recommended by Goodreads paper)

als_model2 = ALS(userCol = "user_id", itemCol = "book_id", ratingCol = "rating",
                 rank = 16, maxIter = 10, regParam = 1,
               nonnegative = True,
               coldStartStrategy = "drop",
               implicitPrefs = False)

model2 = als_model2.fit(train)
predictions2 = model2.transform(test)
evaluator = RegressionEvaluator(metricName = 'rmse', labelCol = 'rating',
                               predictionCol = 'prediction')

rmse2 = evaluator.evaluate(predictions2)
print("RMSE: "), rmse2

RMSE: 


(None, 0.40577705412252574)

In [15]:
predictions2.show()

+-------+-------+------+----------+
|user_id|book_id|rating|prediction|
+-------+-------+------+----------+
|   1645|    148|   4.0|       0.0|
|   3175|    148|   0.0|       0.0|
|   3918|    148|   0.0|       0.0|
|   5300|    148|   0.0|       0.0|
|   1025|    148|   0.0|       0.0|
|   1127|    148|   0.0|       0.0|
|   1507|    148|   0.0|       0.0|
|   2387|    148|   0.0|       0.0|
|   2563|    148|   0.0|       0.0|
|   3475|    148|   0.0|       0.0|
|   4190|    148|   0.0|       0.0|
|   4929|    148|   0.0|       0.0|
|   1143|    148|   0.0|       0.0|
|   3000|    148|   0.0|       0.0|
|    808|    148|   0.0|       0.0|
|   1265|    148|   0.0|       0.0|
|   3098|    148|   0.0|       0.0|
|   4078|    148|   0.0|       0.0|
|   4684|    148|   0.0|       0.0|
|   5223|    148|   0.0|       0.0|
+-------+-------+------+----------+
only showing top 20 rows



In [16]:
param_grid = ParamGridBuilder().addGrid(als_model.rank, [5, 10, 15, 20]).addGrid(
    als_model.maxIter, [5, 10]).addGrid(als_model.regParam, [0.01, 0.05, 0.1, 0.15]).build()
evaluator = RegressionEvaluator(metricName = "rmse", labelCol = "rating",
                               predictionCol = "prediction")
cv = CrossValidator(estimator = als_model,
                   estimatorParamMaps = param_grid,
                   evaluator = evaluator,
                   numFolds = 5)
print ("Num models to be tested: ", len(param_grid))

Num models to be tested:  32


In [17]:
modelcv = cv.fit(train)

In [18]:
best_model = modelcv.bestModel
print(type(best_model))

<class 'pyspark.ml.recommendation.ALSModel'>


In [20]:
test_predictions = best_model.transform(test)
rmse = evaluator.evaluate(test_predictions)
print(rmse)

0.3622062135603919


In [21]:
print(best_model.rank) # k value (# of latent features)

25


In [22]:
print(best_model._java_obj.parent().getMaxIter())
print(best_model._java_obj.parent().getRegParam())

10
0.01


In [ ]:
# best model: k = 25, maxIter = 10, regParam = 0.01, RMSE = 0.3622

In [23]:
test_predictions.show()

+-------+-------+------+-----------+
|user_id|book_id|rating| prediction|
+-------+-------+------+-----------+
|   1645|    148|   4.0| 0.70989245|
|   3175|    148|   0.0| 0.15258428|
|   3918|    148|   0.0| 0.71268547|
|   5300|    148|   0.0|   0.302438|
|   1025|    148|   0.0|   0.245731|
|   1127|    148|   0.0| 0.13814119|
|   1507|    148|   0.0| 0.27556774|
|   2387|    148|   0.0| 0.13265418|
|   2563|    148|   0.0|   0.218646|
|   3475|    148|   0.0| 0.22585842|
|   4190|    148|   0.0| 0.69588304|
|   4929|    148|   0.0| 0.15377276|
|   1143|    148|   0.0| 0.58321583|
|   3000|    148|   0.0|0.012677923|
|    808|    148|   0.0| 0.49552304|
|   1265|    148|   0.0|  0.0973807|
|   3098|    148|   0.0| 0.16174576|
|   4078|    148|   0.0| 0.02927421|
|   4684|    148|   0.0|0.107607946|
|   5223|    148|   0.0|  0.9644963|
+-------+-------+------+-----------+
only showing top 20 rows



In [26]:
# view recommendations
userRecs = best_model.recommendForAllUsers(10)
userRecs.show()

+-------+--------------------+
|user_id|     recommendations|
+-------+--------------------+
|   1580|[[11, 1.0400147],...|
|   5300|[[37, 2.0233123],...|
|   1591|[[167, 0.19397956...|
|   4101|[[11, 1.3917072],...|
|   1342|[[476, 0.63664454...|
|   2122|[[4, 1.1893904], ...|
|    463|[[7, 4.087946], [...|
|    833|[[94, 1.4590017],...|
|   3794|[[168, 1.8770207]...|
|   1645|[[11, 2.7405286],...|
|   3175|[[19, 4.241493], ...|
|   2366|[[205, 4.1261687]...|
|   5156|[[65, 3.4123454],...|
|   3997|[[10, 0.30373782]...|
|   1238|[[11, 3.7416406],...|
|   3918|[[50, 4.152341], ...|
|   4818|[[26, 1.1737348],...|
|   5518|[[125, 1.6194955]...|
|   1829|[[50, 1.9392428],...|
|   3749|[[168, 0.92164904...|
+-------+--------------------+
only showing top 20 rows



In [36]:
# Look at user 60's ratings
print("User 2's Ratings:")
ratings.filter(col("user_id") == 2).sort("rating", ascending = False).show()

# Look at the movies recommended to user 60
print("User 2's Recommendations:")
userRecs.filter(col("user_id") == 2).show()

# Look at user 63's ratings
print("User 63's Ratings:")
ratings.filter(col("user_id") == 63).sort("rating", ascending = False).show()

# Look at the movies recommended to user 63
print("User 63's Recommendations:")
userRecs.filter(col("user_id") == 63).show()

User 60's Ratings:
+-------+-------+------+
|user_id|book_id|rating|
+-------+-------+------+
|      2|    260|   5.0|
|      2|   3753|   5.0|
|      2|   9296|   5.0|
|      2|   8519|   5.0|
|      2|   2686|   5.0|
|      2|    301|   5.0|
|      2|   4081|   4.0|
|      2|     33|   4.0|
|      2|     26|   4.0|
|      2|   2318|   3.0|
|      2|    315|   3.0|
|      2|    471|   0.0|
|      2|    496|   0.0|
|      2|    148|   0.0|
|      2|   1580|   0.0|
|      2|   1238|   0.0|
|      2|   2142|   0.0|
|      2|   2366|   0.0|
|      2|    833|   0.0|
|      2|   3997|   0.0|
+-------+-------+------+
only showing top 20 rows

User 60s Recommendations:
+-------+--------------------+
|user_id|     recommendations|
+-------+--------------------+
|      2|[[11, 0.77537215]...|
+-------+--------------------+

User 63's Ratings:
+-------+-------+------+
|user_id|book_id|rating|
+-------+-------+------+
|     63|    323|   5.0|
|     63|   6772|   5.0|
|     63|    592|   5.0|
|   

In [50]:
exploded_recs = spark.sql("SELECT user_id, explode(recommendations) AS BookRec FROM ALS_recs_temp")
exploded_recs.show()

+-------+----------------+
|user_id|         BookRec|
+-------+----------------+
|   1580| [11, 1.0400147]|
|   1580|[33, 0.65387654]|
|   1580|[100, 0.5300547]|
|   1580|[38, 0.52326894]|
|   1580| [67, 0.4987593]|
|   1580|[57, 0.47979054]|
|   1580|[45, 0.47413552]|
|   1580| [4, 0.46988013]|
|   1580| [22, 0.4662242]|
|   1580|[26, 0.45919847]|
|   5300| [37, 2.0233123]|
|   5300| [58, 1.9057353]|
|   5300| [59, 1.7356001]|
|   5300|  [29, 1.725448]|
|   5300|[138, 1.6259873]|
|   5300| [50, 1.6185813]|
|   5300|[102, 1.6020172]|
|   5300| [15, 1.5831457]|
|   5300|[117, 1.5628898]|
|   5300| [85, 1.5290784]|
+-------+----------------+
only showing top 20 rows

